# Setup

In [ ]:
from ipywidgets import FileUpload, interact

import requests
import pandas as pd
import time

# Autenticação API Github

In [ ]:
@interact(files=FileUpload())
def set_token(files={}):
    global GITHUB_TOKEN
    if files:
        if isinstance(files, tuple): # Jupyter Notebook
            file = files[0]
            token = file['content'].tobytes().decode("utf-8").strip()
        elif isinstance(files, dict): # Google Colab
            file = list(files.values())[0]
            GITHUB_TOKEN = file['content'].decode("utf-8").strip()
        print("Token Loaded!")

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…

In [ ]:
SITE = "https://api.github.com"
def token_auth(request):
    request.headers["User-Agent"] = "MetodosPesquisa" # Required
    request.headers["Authorization"] = "token {}".format(GITHUB_TOKEN)
    return request
response = requests.get(SITE, auth=token_auth)
response.status_code

200

In [ ]:
github_endpoints = response.json()
github_endpoints

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

# Obter nomes de repositórios através dos commits coletados anteriormente

In [ ]:
collected_commits_df = pd.read_csv('/content/merged_commits_11-11-2025_12-57-06.csv')
print(collected_commits_df.shape)
collected_commits_df.head()

(258, 10)


,repo,sha,author,date,message,url,stars,contributors,description,total_commits
0,tolgraven/tolgrAVen,3f5011a0e8f2dd4f260c7d3474215460d9c90548,Joen Tolgraven,2025-09-22T23:38:38.000+02:00,project: remove node-sass,https://github.com/tolgraven/tolgrAVen/commit/...,1.0,1.0,Code for my website at https://tolgraven.se,1264.0
1,dwp/retirement-maintain,0203997f12d501c5437c481733254871c40e9b83,Gemma Leigh,2016-01-12T14:33:49.000Z,Merge pull request #119 from alphagov/remove-n...,https://github.com/dwp/retirement-maintain/com...,1.0,1.0,NaN,7965.0
2,Eddayy/Eddayy.github.io,e1bcc7404a95aca0a3ffec68642107e325d27edc,Edmund Tay,2025-08-25T14:41:58.000+08:00,Merge pull request #105 from Eddayy/renovate/n...,https://github.com/Eddayy/Eddayy.github.io/com...,1.0,1.0,My personal website,278.0
3,AndriySikora/ionic2,ac55ae96cd917c09190c212ab2cb060a3b317826,Andriy Sikora,2025-09-24T11:57:21.000+02:00,Merge pull request #11 from AndriySikora/depen...,https://github.com/AndriySikora/ionic2/commit/...,0.0,1.0,Ionic2 app,14.0
4,enara28/enara_itzala,c248d503b5aef0e66ec6cc2fdeb3d4712b2bd8b9,enara28,2025-09-18T12:32:22.000+02:00,cambio node-sass por sass,https://github.com/enara28/enara_itzala/commit...,0.0,0.0,NaN,16.0


In [ ]:
repos = collected_commits_df['repo'].unique().tolist()
len(repos)

178

# Coletar dados dos repositórios listados

In [ ]:
# collected_commits_df_old = pd.read_excel('/content/commits_projetos.xlsx')
# repos_old = collected_commits_df_old['repo'].unique().tolist()
# len(repos_old)

121

In [ ]:
def get_all_keys(d):
    keys = []
    for key, value in d.items():
        keys.append(key)
        if isinstance(value, dict):
            keys.extend(get_all_keys(value))
    return keys


def flatten_dict(nested_dict, parent_key='', sep='/') -> dict:
    items = []
    for key, value in nested_dict.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key

        if isinstance(value, dict):
            items.extend(flatten_dict(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))

    return dict(items)

In [ ]:
all_repos_info = []

for repo in repos:
    owner_name, repo_name = repo.split("/")
    # print(owner_name, repo_name)
    repo_response_raw = requests.get(github_endpoints['repository_url'].format(owner=owner_name, repo=repo_name), auth=token_auth)
    # print(repo_response_raw.status_code)
    repo_response = repo_response_raw.json()

    all_keys = get_all_keys(repo_response)
    repo_info = flatten_dict(repo_response)
    repo_info["all_keys"] = all_keys
    all_repos_info.append(repo_info)

repo_df = pd.DataFrame(all_repos_info)
repo_df.shape

(178, 432)

**Obs.:** Às vezes retorna 432 colunas, outras vezes retorna 238.

In [ ]:
repo_df.head(1)

,id,node_id,name,full_name,private,owner/login,owner/id,owner/node_id,owner/avatar_url,owner/gravatar_id,...,template_repository/visibility,template_repository/forks,template_repository/open_issues,template_repository/watchers,template_repository/default_branch,template_repository/permissions/admin,template_repository/permissions/maintain,template_repository/permissions/push,template_repository/permissions/triage,template_repository/permissions/pull
0,298904100.0,MDEwOlJlcG9zaXRvcnkyOTg5MDQxMDA=,tolgrAVen,tolgraven/tolgrAVen,False,tolgraven,9143357.0,MDQ6VXNlcjkxNDMzNTc=,https://avatars.githubusercontent.com/u/914335...,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
repo_df.shape

(178, 432)

In [ ]:
new = repo_df.copy()
new.shape

(173, 238)

In [ ]:
sos = repo_df.copy()
sos.shape

(118, 432)

# Salvar dados coletados

In [ ]:
print(repo_df.shape)
repo_df.to_csv("repositories_info.csv", index=False)

(178, 432)


# Pré-processamento dos dados dos repositórios

## Verificando retornos vazios

In [ ]:
repo_df["full_name"].isna().sum()

np.int64(3)

In [ ]:
[lost_repo for lost_repo in repos if lost_repo not in repo_df["full_name"].values.tolist()]

['enara28/enara_itzala', 'darskkaa/sneakerbots', 'Mrserg123/test-deploy']

**3 repositórios não estão mais disponíveis**. Eles podem ter sido removidos ou sofreram restrições de acesso público.

In [ ]:
print(repo_df.shape)
repo_df.dropna(subset="full_name", inplace=True)
print(repo_df.shape)

(178, 432)
(175, 432)


## Remoção de colunas indesejadas

In [ ]:
repo_df.columns

Index(['id', 'node_id', 'name', 'full_name', 'private', 'owner/login',
       'owner/id', 'owner/node_id', 'owner/avatar_url', 'owner/gravatar_id',
       ...
       'template_repository/visibility', 'template_repository/forks',
       'template_repository/open_issues', 'template_repository/watchers',
       'template_repository/default_branch',
       'template_repository/permissions/admin',
       'template_repository/permissions/maintain',
       'template_repository/permissions/push',
       'template_repository/permissions/triage',
       'template_repository/permissions/pull'],
      dtype='object', length=432)

In [ ]:
owner_cols = [col for col in repo_df.columns.tolist() if col.startswith("owner")] # owner/type
license_cols = [col for col in repo_df.columns.tolist() if col.startswith("license")]
template_cols = [col for col in repo_df.columns.tolist() if col.startswith("template_repository")]
permission_cols = [col for col in repo_df.columns.tolist() if col.startswith("permissions")]

cols_to_drop = [
    'id', 'node_id', 'private', 'name',
    'keys_url', 'teams_url', 'hooks_url', 'blobs_url' , 'git_tags_url', 'git_refs_url' , 'trees_url',
    'statuses_url', 'collaborators_url', 'issue_events_url',
    'git_url', 'ssh_url', 'clone_url', 'svn_url', 'homepage', 'watchers_count', 'license',
    'allow_forking', 'is_template', 'visibility',
    'commits_url', 'git_commits_url', 'comments_url', 'issue_comment_url', 'compare_url',
    'contents_url', #
    'archive_url', 'issues_url', 'milestones_url', 'releases_url', 'labels_url', 'notifications_url',
    'pulls_url',
    'all_keys' # coluna de segurança, deixar apenas na versão full



]
cols_to_drop.extend(owner_cols)
cols_to_drop.extend(license_cols)
cols_to_drop.extend(template_cols)
cols_to_drop.extend(permission_cols)

In [ ]:
repo_df.shape, len(cols_to_drop)

((175, 432), 174)

In [ ]:
repo_df_filtered = repo_df.drop(columns=cols_to_drop)
repo_df.shape, repo_df_filtered.shape

((175, 432), (175, 259))

In [ ]:
repo_df_filtered.to_csv("repositories_info_reduced.csv", index=False)

In [ ]:
repo_df_filtered.isna().sum()

,0
full_name,0
html_url,0
description,76
fork,0
url,0
...,...
source/visibility,174
source/forks,174
source/open_issues,174
source/watchers,174


## Separando dados de fork

In [ ]:
repo_df_filtered["fork"].value_counts()

,count
fork,
False,174
True,1


In [ ]:
fork_df = repo_df_filtered[repo_df_filtered["fork"]].copy()
fork_df.shape

(1, 259)

In [ ]:
fork_df

,full_name,html_url,description,fork,url,forks_url,events_url,assignees_url,branches_url,tags_url,...,source/license,source/allow_forking,source/is_template,source/web_commit_signoff_required,source/topics,source/visibility,source/forks,source/open_issues,source/watchers,source/default_branch
5,CharlieHammersley/raptors1711.com,https://github.com/CharlieHammersley/raptors17...,The official website for FRC team 1711 - The R...,True,https://api.github.com/repos/CharlieHammersley...,https://api.github.com/repos/CharlieHammersley...,https://api.github.com/repos/CharlieHammersley...,https://api.github.com/repos/CharlieHammersley...,https://api.github.com/repos/CharlieHammersley...,https://api.github.com/repos/CharlieHammersley...,...,NaN,True,False,False,"[frc, react, website]",public,3.0,9.0,0.0,master


In [ ]:
common_df = repo_df_filtered[repo_df_filtered["fork"] == False].copy()
common_df.shape

(174, 259)

## Limpando dados de repositórios comuns (sem fork)

In [ ]:
repo_df_filtered["parent/id"].value_counts()

,count
parent/id,
428454444.0,1


In [ ]:
repo_df_filtered["source/id"].value_counts()

,count
source/id,
428454444.0,1


In [ ]:
fork_cols = [col for col in repo_df.columns.tolist() if col.startswith("source") or col.startswith("parent")]
fork_cols.extend(["forks_url"])

common_df_filtered = common_df.drop(columns=fork_cols)
common_df.shape, common_df_filtered.shape


((174, 259), (174, 64))

## Salvando dados de repositórios após ajustes

In [ ]:
common_df_filtered.head(1)

,full_name,html_url,description,fork,url,events_url,assignees_url,branches_url,tags_url,languages_url,...,organization/organizations_url,organization/repos_url,organization/events_url,organization/received_events_url,organization/type,organization/user_view_type,organization/site_admin,message,documentation_url,status
0,tolgraven/tolgrAVen,https://github.com/tolgraven/tolgrAVen,Code for my website at https://tolgraven.se,False,https://api.github.com/repos/tolgraven/tolgrAVen,https://api.github.com/repos/tolgraven/tolgrAV...,https://api.github.com/repos/tolgraven/tolgrAV...,https://api.github.com/repos/tolgraven/tolgrAV...,https://api.github.com/repos/tolgraven/tolgrAV...,https://api.github.com/repos/tolgraven/tolgrAV...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
fork_df.to_csv("forked_repos.csv", index=False)
# common_df_filtered.to_csv("repositories_info_reduced_no_fork.csv", index=False)
common_df_filtered.to_csv("common_repos.csv", index=False)